# Simple Wall Example

In this example, we show how to extend the simple Agent to be blocked by a wall.

In [2]:
import os, sys, curses, torch, six, itertools, collections
import numpy as np

from campx import things
from campx.ascii_art import ascii_art_to_game, Partial
from campx import engine

In [3]:
GAME_ART = ['#####',
             '#A* #',
             '#*#*#',
             '# * #',
             '#####']

In [10]:
class AgentDrape(things.Drape):
    """A Drape that just moves an agent around the board using a probablility vector"""
    
    def __init__(self, curtain, character, blocking_chars="#"):
        super(AgentDrape, self).__init__(curtain, character)
        
        self.blocking_chars = blocking_chars
    
    def update(self, actions, board, layers, backdrop, all_things, the_plot):
        del board, backdrop, all_things  # unused

        if actions is not None:

            act = actions

            b = self.curtain

            left = torch.cat([b[:,1:],b[:,:1]], dim=1)
            right = torch.cat([b[:,-1:],b[:,:-1]], dim=1)
            up= torch.cat([b[1:],b[:1]], dim=0)
            down = torch.cat([b[-1:],b[:-1]], dim=0)
            stay = b

            b = (act[0] * left) + (act[1] * right) + (act[2] * up) + (act[3] * down) + (act[4] * stay)

            # Does this move overlap with a blocking character?
            for c in self.blocking_chars:
                if('prev_pos_'+self.character in the_plot):
                    gate = (b * (1 - layers[c])).sum() # 1 if not going behind wall, # 0 otherwise
                    b = (gate * b) + (the_plot['prev_pos_'+self.character] * (1 - gate))

            self.curtain.set_(b)

            the_plot.add_reward(1)  # Give ourselves a point for moving.

        # cache previous position for use later
        the_plot['prev_pos_'+self.character] = layers[self.character]



In [11]:
def make_game():
  """Builds and returns a Hello World game."""
  return ascii_art_to_game(
      GAME_ART,
      what_lies_beneath=' ',
      drapes={'A': AgentDrape,
             '#': things.FixedDrape},
      z_order='A#')
game = make_game()

In [12]:
board, reward, discount = game.its_showtime()

In [15]:
# this will move the 65 around the board ([left, right, up, down, stay])
# run it multiple times. Notice how the "65" is blocked by all "35" items

board, reward, discout = game.play([1,0,0,0,0])

board.board


 35  35  35  35  35
 35  65   0   0  35
 35   0  35   0  35
 35   0   0   0  35
 35  35  35  35  35
[torch.LongTensor of size 5x5]